In [1]:
import damask
import numpy as np
import yaml

/home/snkrkr/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/snkrkr/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/home/snkrkr/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# Example: Generate single phase single material polycrystal material config

In [2]:
with open('./examples/config/phase/mechanical/elastic/Hooke_al.yaml', 'r') as f:
    al_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/phenopowerlaw_Al.yaml', 'r') as f:
    al_plastic = yaml.safe_load(f)

In [5]:
mat = damask.ConfigMaterial()

homogenization = {
    'SX': {
        'N_constituents': 1,
        'mechanical': {'type': 'pass'} # No volume fractions, so no homogenization needed
    }
}
mat.update(homogenization=homogenization)

phase_alu = {
    'Aluminium':{
        'lattice': 'cF', # FCC material
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': al_elastic,
            'plastic': al_plastic
        }
    }
}
mat.update(phase=phase_alu)

N_grains = 10
rotations = damask.Rotation.from_random(N_grains)
mat = mat.material_add(O=rotations,phase='Aluminium',homogenization='SX')
mat.is_complete

True

In [6]:
mat.save(f'./material_Al_{N_grains}.yaml')

# Example: Generate DP Steel with distinct grains

In [34]:
with open('./examples/config/phase/mechanical/elastic/Hooke_Fe.yaml', 'r') as f:
    martensite_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/phenopowerlaw_bcc-martensite.yaml', 'r') as f:
    martensite_plastic = yaml.safe_load(f)

with open('./examples/config/phase/mechanical/elastic/Hooke_Fe.yaml', 'r') as f:
    ferrite_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/phenopowerlaw_DP-steel-ferrite.yaml', 'r') as f:
    ferrite_plastic = yaml.safe_load(f)

In [35]:
mat = damask.ConfigMaterial()

homogenization = {
    'SX': {
        'N_constituents': 1,
        'mechanical': {'type': 'pass'} # No volume fractions, so no homogenization needed
    }
}
mat.update(homogenization=homogenization)

phase_martesite = {
    'Martensite':{
        'lattice': 'cI', # BCC
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': martensite_elastic,
            'plastic': martensite_plastic
        }
    }
}

phase_ferrite = {
    'Ferrite':{
        'lattice': 'cI', # BCC
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': ferrite_elastic,
            'plastic': ferrite_plastic
        }
    }
}

mat.update(phase=phase_martesite|phase_ferrite)

N_grains_ferrite = 6
rotations = damask.Rotation.from_random(N_grains)
mat = mat.material_add(O=rotations,phase='Ferrite',homogenization='SX')

N_grains_martensite = 4
rotations = damask.Rotation.from_random(N_grains)
mat = mat.material_add(O=rotations,phase='Martensite',homogenization='SX')

mat.is_complete

True

In [36]:
mat.save(f'material_fullfield_{N_grains_ferrite}ferrite_{N_grains_martensite}martensite.yaml')

# Example: Generate DP steel with volume fraction

In [30]:
with open('./examples/config/phase/mechanical/elastic/Hooke_Fe.yaml', 'r') as f:
    martensite_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/phenopowerlaw_bcc-martensite.yaml', 'r') as f:
    martensite_plastic = yaml.safe_load(f)

with open('./examples/config/phase/mechanical/elastic/Hooke_Fe.yaml', 'r') as f:
    ferrite_elastic = yaml.safe_load(f)
with open('./examples/config/phase/mechanical/plastic/phenopowerlaw_DP-steel-ferrite.yaml', 'r') as f:
    ferrite_plastic = yaml.safe_load(f)

In [32]:
N_materials = 10
martensite_fraction = 0.4
ferrite_fraction = 0.6

mat = damask.ConfigMaterial()

homogenization = {
    'Taylor': {
        'N_constituents': 2,
        'mechanical': {'type': 'isostrain'} # Volume fractions present, so homogenization needed
    }
}
mat.update(homogenization=homogenization)

phase_martesite = {
    'Martensite':{
        'lattice': 'cI', # BCC
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': martensite_elastic,
            'plastic': martensite_plastic
        }
    }
}

phase_ferrite = {
    'Ferrite':{
        'lattice': 'cI', # BCC
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': ferrite_elastic,
            'plastic': ferrite_plastic
        }
    }
}

mat.update(phase=phase_martesite|phase_ferrite)

mat = mat.material_add(phase = ['Martensite', 'Ferrite'],
                       O = damask.Rotation.from_random((N_materials,2)),
                       v = np.array([[martensite_fraction]+[ferrite_fraction]]),
                       homogenization = 'Taylor')
mat.is_complete

True

In [33]:
mat.save(f'material_volumefraction_{N_materials}_{ferrite_fraction}ferrite_{martensite_fraction}martensite.yaml')

## Exercises:

- Generate a single-phase IF-steel material with 50 distinct materials (grains), using the dislotwin constitutive law.
  - Bonus: generate a corresponding gridfile and run the simulation
- Generate a dual-phase steel with 20 distinct materials and a volume fraction of 50% each